Timer start

In [ ]:
from time import perf_counter

start_script_time = perf_counter()

# Init Block

In [ ]:
# imports
from datetime import date, timedelta, datetime
from time import sleep
from pathlib import Path
import re
import logging
from dotenv import dotenv_values
import requests
import ast
from typing import Any

import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values, DictCursor

import gspread
from google.oauth2.service_account import Credentials

import smtplib
import ssl
from email.message import EmailMessage

ENV_FILE=dotenv_values('.env')

# -------------Bamboo PostgreSQL DB config-------------
BAMBOO_DB_DSN = ENV_FILE.get('DB_DSN_bamboo')

# -------------Google Sheets config-------------
GS_CREDS_FILE = Path('bamboo_service_account.json')  # path to JSON google key

DASHBOARD_SPREADSHEET_ID = "1U5ApJgj2_4CWYd_7nV1_LUvUL1aKmU8dB96OlLSd170"      # id from URL
DASHBOARD_WORKSHEET_NAME = "Лист1"                         # list name
DASHBOARD_SCOPES = ["https://www.googleapis.com/auth/spreadsheets"] # rights request

REQUESTS_PER_MIN_LIMIT = 50
SLEEP_BETWEEN_REQUESTS_SEC = 65

# -------------API and download period config-------------

# today time period
# DOWNLOAD_TIME_START = f"{date.today()} 00:00:00.000000"
# DOWNLOAD_TIME_END = f"{date.today()} 23:59:59.999999"

#fixed time period 
DOWNLOAD_TIME_START = '2025-01-01 00:00:00.000000'
DOWNLOAD_TIME_END = '2025-06-30 23:59:59.999999'

ITRESUME_URL = "https://b2b.itresume.ru/api/statistics"
ITRESUME_SKILLFACTORY_PARAMS={
'client': 'Skillfactory',
'client_key':ENV_FILE.get('CLIENT_KEY_Skillfactory'),
'start':DOWNLOAD_TIME_START,
'end':DOWNLOAD_TIME_END
}

# -------------GMAIL config-------------
BARROCO_SMTP = 'smtp.gmail.com'
BARROCO_EMAIL_PORT= 465
BARROCO_ACCOUNT_PASSWORD = ENV_FILE.get('GOOGLE_APP_PASS')
BARROCO_EMAIL = 'unpocobarroco@gmail.com'
BARROCO_EMAIL_NAME = 'Bamboo Project'

# DEFINITIONS

### Classes

- **DB_Ops**: communication with DB
- **Mail_Server_SSL**: communication with Mail-sevice Server
    - SSL connection with SMTP
    - Use `with...as` to ensure SMTP-session is closed

In [ ]:
class DbOps:
    """
    - Encapsulation tools for communication with PostgreSQL database. 
    - 1 instance = 1 target database (connections are created per operation).
    - Takes DSN (Data Source Name) as a string. Example: "dbname=name user=user password=secret host=localhost port=5432".
    """
    
    def __init__(self, db_dsn:str ):
        self.dsn=db_dsn
        
    def make_table(self, schema:str, table:str, creating_query:str):
        """Make the new table on the bases of received query. 
        'with...as' construction ensures closing connection in case of success and rollback if failed.
        
        Example:
        CREATE TABLE IF NOT EXISTS {} (
        id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY
        , user_id TEXT NOT NULL
        , oauth_consumer_key TEXT
        , created_at TIMESTAMP NOT NULL
        , CONSTRAINT unique_check UNIQUE (user_id, created_at)
        )

        Args:
            schema (str): new table needed schema
            table (str): new table name
            creating_query (str): full SQL query
        """
        with psycopg2.connect(self.dsn) as conn:
            with conn.cursor() as cur:
                
                ready_query=sql.SQL(creating_query).format(sql.Identifier(schema, table))
                cur.execute(ready_query)
    
    @staticmethod
    def make_sql_columns(columns:list|tuple):
        """Safely compose a dynamic SQL fragment using psycopg2.sql.

        Args:
            columns: list or tuple

        Returns: 
            Composed SQL fragment
            
        """
        return sql.SQL(', ').join(sql.Identifier(col) for col in columns)
    
    
    def insert_unique_data_to_table(self, schema:str, table:str, unique_constraint:str, columns:list|tuple, data:list|tuple):
        """Safely inserts data into the specified columns. If data is repeated, it will be skipped (ON CONFLICT ON CONSTRAINT ... DO NOTHING).
        'with...as' construction ensures closing connection in case of success and rollback if failed.

        Args:
            schema (str): needed schema
            table (str): table name
            unique_constraint (str): name of the unique constraint in the target table.
            columns: list or tuple
            data: sequence of rows

        Returns: 
            int: Number of inserted rows
                  
        """
        
        if not data:
            return 0
        
        needed_length=len(columns)
        for i, row in enumerate(data):
            if len(row)!=needed_length:
                raise ValueError(f"{i} row length is not the same as columns count in DB table ({needed_length})")
        
        columns_sql=DbOps.make_sql_columns(columns)
        
        with psycopg2.connect(self.dsn) as conn:
            with conn.cursor() as cur:
        
                query=sql.SQL(
                """
                INSERT INTO {} ({})
                VALUES %s
                ON CONFLICT ON CONSTRAINT {} DO NOTHING
                RETURNING 1
                """).format(sql.Identifier(schema, table), columns_sql, sql.Identifier(unique_constraint))
                
                returned = execute_values(cur, query, data, page_size=1000, fetch=True)
                return len(returned)
                
            
    def query_fetchall_dict(self, query:str):
        """
        - Executing-fetch function for general SQL queries.
        - 'with...as' construction ensures closing connection in case of success and rollback if failed.
        - No {} or %s are allowed in query

        Args:
            query (str): full SQL query

        Returns:
            list[DictRow,...]: List of dict-like rows (DictRow). 1 DictCursor = 1 row in the result table.
        """

        with psycopg2.connect(self.dsn) as conn:
            with conn.cursor(cursor_factory=DictCursor) as cur:
                
                cur.execute(query)
                return cur.fetchall()


class MailServerSSL:
    """
    - Encapsulates sending email via SMTP over SSL using provided account credentials.
    - 1 instance = 1 email account
    - Takes smtp_server:str, ssl_port:int, account_password:str, sender_email:str, sender_name:str.
    """
    
    context = ssl.create_default_context()
    
    def __init__(self, smtp_server:str, ssl_port:int, account_password:str, sender_email:str, sender_name:str):
        self.account_password=account_password
        self.smtp_server = smtp_server
        self.ssl_port = ssl_port
        self.sender_email = sender_email
        self.sender_name=sender_name
        
    def send_message(self, recipient_email:str, subject:str, message:str):
        """Send a plain text email.

        Args:
            recipient_email (str): mail to
            subject (str): subject of the message
            message (str): message text

        Returns:
            EmailMessage: Mail message object
        """

        msg = EmailMessage()
        
        msg['Subject'] = subject
        msg['From'] = f"{self.sender_name} <{self.sender_email}>"
        msg['To'] = recipient_email
        msg.set_content(message)
        
        
        with smtplib.SMTP_SSL(self.smtp_server, self.ssl_port, context=MailServerSSL.context) as server:
            server.login(self.sender_email, self.account_password)
            server.send_message(msg)
            
        return msg
    

class GoogleSpreadsheet:
    """
    - Encapsulation tools for communication with Google Spreadsheet. 
    - 1 instance = 1 Spreadsheet in target account.
    - Takes creds_file: Path, scopes: list[str], spreadsheet_id: str.
    
    Scopes example: ["https://www.googleapis.com/auth/spreadsheets"]
    spreadsheet_id: get from URL in browser
    
    """
        
    def __init__(self, creds_file: Path, scopes: list[str], spreadsheet_id: str):
        
        self.creds = Credentials.from_service_account_file(creds_file, scopes=scopes)
        self.gc = gspread.authorize(self.creds)         # client
        self.sh = self.gc.open_by_key(spreadsheet_id)   # spreadsheet
        self.ws = None                                  # worksheet

    def select_worksheet(self, worksheet_name: str):
        """
        - Return a worksheet by name, reusing the cached worksheet if available.
        - If the cached worksheet title matches the requested name, it is returned directly.

        Args:
            worksheet_name (str): Name of the worksheet (not id)

        Returns: gspread.Worksheet
        """
        
        if self.ws and self.ws.title == worksheet_name:
            return self.ws
        
        self.ws = self.sh.worksheet(worksheet_name)
        return self.ws

###  Helper functions

In [ ]:
def cleanup_logs(log_dir:Path, keep_limit_days:int):
    """
    - Cleans up the old files in target folder.
    - Files must contain date in the filename ("yyyy-mm-dd").

    Args:
        log_dir (Path): path object of the target directory
        keep_limit_days (int): delete files older then
    """

    today_date=date.today()
    keep_limit_date=today_date-timedelta(days=keep_limit_days)

    log_dir.mkdir(parents=True, exist_ok=True)

    for p in log_dir.iterdir():
        
        if not p.is_file():
            continue
        
        filename=p.name
        match_date_in_name=re.search(r"\d{4}-\d{2}-\d{2}",filename)
        
        if not match_date_in_name:
            continue
        iso_date_in_name=date.fromisoformat(match_date_in_name.group())
        
        if iso_date_in_name > keep_limit_date:
            continue
        
        p.unlink()
   
        
def get_periods(total_seconds:float|int):
    """Extracts day, hour, minute, and second components from a duration in seconds.

    Args:
        total_seconds (float | int): number of seconds

    Returns:
        dict: Extracted time periods including zero values.
    """

    days, reminder = divmod(total_seconds, 86400)
    hours, reminder = divmod(reminder, 3600)
    mins, secs = divmod(reminder, 60)

    return {'d': days, 'h': hours, 'm': mins, 's': secs}
        

def string_to_dict(string:str):
    """Converts string to dict format.

    Args:
        string (str): string

    Returns:
        dict: dict with content (if conversion succeeded) or empty dict (if failed)
    """
    
    try:
        res = ast.literal_eval(string)
        
        if not isinstance(res, dict):
            logging.warning("Converted object format is not dict")
            return {}           
        
        return res
        
    except Exception:
        logging.warning(f"Not able to convert passback_params to dict format")
        return {}
        

def convert_to_datetime(string:str):
    """Validates the string content: returns datetime object if iso-format, if fail - raise ValueError.

    Args:
        string (str): string

    Raises:
        ValueError: "Date conversion Error"

    Returns:
        datetime: Datetime object
    """
    
    try:
        return datetime.fromisoformat(string)
    
    except Exception:
        logging.warning(f"Not able to convert {string} to ISO-format string.")
        raise ValueError("Date conversion Error")


def validate_return_value(value:Any, accepted_vals: tuple | None = None, class_or_tuple = None):
    """Validates the value type and allowed values. Raises Validation Error if fails.

    Args:
        value (Any): value
        accepted_vals (tuple | None, optional): accepted vals tuple. Defaults to None.
        class_or_tuple (_type_, optional): expected type or tuple of types. Defaults to None.

    Raises:
        ValueError: "Value Validation Error" or "Type Validation Error"

    Returns:
        Any: Target value (if the validation succeeds)
    """
    
    if class_or_tuple is not None and not isinstance(value, class_or_tuple):
        logging.warning(f"Type Validation Error: {value} doesn`t fit {class_or_tuple}")
        raise ValueError("Type Validation Error")
    
    if accepted_vals is not None and (value not in accepted_vals):
        logging.warning(f"Value Validation Error: {value} is not in {accepted_vals}")
        raise ValueError ("Value Validation Error")
    
    return value


def hook(obj: dict):
    """JSON hook that transforms data to the target structure and validates contained values.

    Args:
        obj (dict): JSON payload received from the webhook.

    Returns:
        dict: Normalized payload as a plain dictionary, or None on failure.
    """
    try:
        
        passback = string_to_dict(obj['passback_params'])
        
        # extract the vals according the correct structure as it is
        user_id = obj['lti_user_id']
        oauth_consumer_key = passback.get('oauth_consumer_key', None)
        lis_result_sourcedid = passback.get('lis_result_sourcedid', None)
        lis_outcome_service_url = passback.get('lis_outcome_service_url', None)
        is_correct = obj['is_correct']
        attempt_type = obj['attempt_type']
        created_at = obj['created_at']
        
        # validate and put to the structure
        plain_dict = {
            'user_id':validate_return_value(user_id, class_or_tuple=str), # строковый айди пользователя
            'oauth_consumer_key': validate_return_value(oauth_consumer_key, class_or_tuple=(str, type(None))), # уникальный токен клиента
            'lis_result_sourcedid': validate_return_value(lis_result_sourcedid, class_or_tuple=(str, type(None))), # ссылка на блок, в котором находится задача в ЛМС
            'lis_outcome_service_url': validate_return_value(lis_outcome_service_url, class_or_tuple=(str, type(None))), # URL адрес в ЛМС, куда мы шлем оценку
            'is_correct': validate_return_value(is_correct, accepted_vals=(0, 1, None), class_or_tuple=(int, type(None))), # была ли попытка верной (null, если это run)
            'attempt_type': validate_return_value(attempt_type, accepted_vals=('run', 'submit'), class_or_tuple=str), # ран или сабмит
            'created_at': convert_to_datetime(created_at)  # дата и время попытки
            }
        
        return plain_dict
    
    except Exception:
        logging.warning(f"Above fail araised in: created_at {obj.get('created_at', 'Empty')}, user_id {obj.get('lti_user_id', 'Empty')}")
        
        return None

# Bootstrap
- Log directory manage
- Log files cleanup
- Logging settings: force=True - гарантия создания нового файла при смене даты

In [ ]:
log_dir = Path("logs")
keep_limit_days=3

cleanup_logs(log_dir, keep_limit_days)

log_file = log_dir / f"logs ({date.today()}).txt"
logging.basicConfig(filename=log_file,level=logging.INFO, filemode='a', format="%(asctime)s %(name)s %(levelname)s: %(message)s", force=True)

# ETL

## Extracting API

### Downloading data by API
- using extra visual check (first_row_data) for further steps

In [ ]:
logging.info("====================== New ETL started ======================")
logging.info("Start downloading data by API")

try:
    
    r=requests.get(ITRESUME_URL, params=ITRESUME_SKILLFACTORY_PARAMS, timeout=None)
    
    r.raise_for_status()
    first_row_data=r.json()
    rows_income_cnt=len(first_row_data)
    
    logging.info(f"Downloading data by API completed successfully. Code {r.status_code}. {rows_income_cnt} rows total")
    
except Exception:
    logging.exception("Invalid get-parameters or read timed out")
    raise


first_row_data

## Data transform

### Validate and transform of downloaded data

In [ ]:
logging.info("----- Raw data transformation started -----")    

try:    
    
    user_solutions_structured=r.json(object_hook=hook)
    logging.info("Transformation stage 1 (rows transformation and validation) completed")   
    
except Exception:
    logging.exception("Transformation stage 1 (rows transformation and validation) failed")
    raise

user_solutions_structured

### Get rid of the skipped rows ("None`s") in the list of user solutions 
- clearing nearly in-place to safe RAM

In [ ]:
try:

    user_solutions_structured[:]=[x for x in user_solutions_structured if x is not None]
    logging.info("Transformation stage 2 (non-working rows clearing) completed")

    rows_transformed_cnt=len(user_solutions_structured)
    
    logging.info(f"All stages of raw data transformation completed: {rows_transformed_cnt} rows of {rows_income_cnt} done")

except Exception:
    logging.exception("Transformation stage 2 (non-working rows clearing) failed")
    raise


user_solutions_structured

## Load to Database

### Creating a table in DB if doesn`t exist + put the info into table

In [ ]:
logging.info("----- Start communication with Database -----")

try:

    bamboo=DbOps(BAMBOO_DB_DSN)

    table_schema ='public'
    table_name = 'solutions'
    unique_constraint = 'unique_check'
    target_in_log=f"{table_schema}.{table_name}"
    
    creating_query="""
    CREATE TABLE IF NOT EXISTS {} (
        id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY
        , user_id TEXT NOT NULL
        , oauth_consumer_key TEXT
        , lis_result_sourcedid TEXT
        , lis_outcome_service_url TEXT
        , is_correct INT CHECK (is_correct in (0, 1) OR is_correct is NULL)
        , attempt_type TEXT CHECK (attempt_type in ('run', 'submit')) NOT NULL
        , created_at TIMESTAMP NOT NULL
        , CONSTRAINT unique_check UNIQUE (user_id, created_at)
    )
    """
    
    bamboo.make_table(table_schema, table_name, creating_query)
    logging.info(f"[{target_in_log}] Checking/creating table completed successfully")
    
    columns_for_inserting=('user_id', 'oauth_consumer_key', 'lis_result_sourcedid', 'lis_outcome_service_url', 'is_correct', 'attempt_type', 'created_at')
      
    tuples_to_insert=[(
        row["user_id"],
        row["oauth_consumer_key"],
        row["lis_result_sourcedid"],
        row["lis_outcome_service_url"],
        row["is_correct"],
        row["attempt_type"],
        row["created_at"]
    ) for row in user_solutions_structured]
    rows_prepared_cnt = len(tuples_to_insert)
    logging.info(f"[{target_in_log}] Insert-tuples list preparing completed successfully") 
    
    inserted_rows_cnt = bamboo.insert_unique_data_to_table(table_schema, table_name, unique_constraint, columns_for_inserting, tuples_to_insert)
    logging.info(f"[{target_in_log}] Inserting data to DB-table completed. {inserted_rows_cnt} new rows of {rows_prepared_cnt} prepared were inserted")
    
    logging.info("Communication with Database completed successfully") 

except Exception:
    logging.exception("Communication with Database failed. Transaction (if started) was rolled back automatically.")
    raise

## SQL Analytics

### Prepare aggregated data dict, using DB
- Always aggregate all data because:
    1) the previous data can be added the following day because of the timezone
    2) worksheet can be changed by everyone, BD - can`t, so need to update correct info

In [ ]:
logging.info("----- Direct database analytics starts -----")

try:    

	db_dic_list= bamboo.query_fetchall_dict(
	""" 
	WITH 
		first_seen AS (
			SELECT 
				min(date(created_at)) AS study_date
				, user_id
			FROM public.solutions
			GROUP BY user_id
		),
		
		new_users AS (
			SELECT
				study_date
				, count(user_id) AS new_users_cnt
			FROM first_seen
			GROUP BY study_date
		),

		solutions_metrics AS (
			SELECT 
				date(created_at) AS study_date
				, count(*) AS attempts_per_day_total
				, count(*) FILTER(WHERE is_correct=1) AS correct_attempts_per_day
				, count(DISTINCT user_id) AS unique_users_per_day	
			FROM public.solutions
			GROUP BY study_date
			ORDER BY study_date
		)
			
	SELECT 
		to_char(sm.study_date, 'yyyy-mm-dd') AS study_date
		, attempts_per_day_total
		, correct_attempts_per_day
		, unique_users_per_day	
		, COALESCE(nu.new_users_cnt, 0) AS new_users_per_day
	FROM solutions_metrics sm
	LEFT JOIN new_users nu USING(study_date)
	ORDER BY study_date
	"""
	)
	
	fetch_rows_cnt = len(db_dic_list)
	logging.info(f"Aggregated report ({fetch_rows_cnt} rows) for Google Spreadsheets Dashboard was successfully fetched by using SQL query to DB")

except Exception:
    logging.exception("SQL query construction failed")
    raise


db_dic_list

## Communucation with Google Spreadsheet

### Sheet headers prepare directly in WS
- get existing Google ws headers, 
- check if headers are the same as in DB, add new headers from DB if needed (nothing deletes)

In [ ]:
logging.info("----- Start communucation with Google Spreadsheet -----")

try:
     
    ws = GoogleSpreadsheet(GS_CREDS_FILE, DASHBOARD_SCOPES, DASHBOARD_SPREADSHEET_ID).select_worksheet(DASHBOARD_WORKSHEET_NAME)
    
    logging.info("Connecting to Google Spreadsheet is stable")

except Exception:
    logging.exception("Using of Google Sheets credentials failed")

try:
    
    if fetch_rows_cnt==0:
        logging.warning("No rows for dashboard. Skipping Worksheet update")
        
    else:
        ws_headers=ws.row_values(1)
        query_headers=db_dic_list[0].keys()

        add_headers=[header for header in query_headers if header not in ws_headers]
            
        if add_headers:
            ws_headers.extend(add_headers)
            ws.update(range_name='1:1', values=[ws_headers], value_input_option="USER_ENTERED")
            
        logging.info("Worksheet headers preparing completed successfully")

except Exception:
    logging.exception("Headers preparing in Worksheet failed")
 
        
ws_headers

### Add aggregated data to WS in Google Sheets
- If date exists- row replace, if not - row append. 
- Can find the date-column dinamicaly. 
- The row structure automatically fits the Google ws neww structure.
- Lines qty doesn`t validate
- Columns width doesn`t change
- Columns with same name doesn`t validate - just change both

In [ ]:
try:
    
    dates_in_ws=ws.col_values(ws_headers.index('study_date')+1)

    for i, dic in enumerate(db_dic_list):
        
        if i>0 and i % REQUESTS_PER_MIN_LIMIT==0:
            logging.info(f"Quota of 'Write requests' exceeded limit {REQUESTS_PER_MIN_LIMIT}. Sleeping {SLEEP_BETWEEN_REQUESTS_SEC} seconds")
            sleep(SLEEP_BETWEEN_REQUESTS_SEC)
        
        row_to_add=[dic.get(key) for key in ws_headers]
        row_date=dic['study_date']
        
        if row_date in dates_in_ws:
            ws.update(range_name=f"A{dates_in_ws.index(row_date)+1}", values=[row_to_add], value_input_option="USER_ENTERED")
            
        else:
            ws.append_row(row_to_add, value_input_option="USER_ENTERED")
            
    logging.info("Aggregated analytics appending to Google Worksheet completed successfully")
    logging.info("Communucation with Google Spreadsheet completed successfully")

except Exception:
    logging.exception("Failed to append analytical report data to Google Worksheet")
    raise

## Communucation with SMTP Server

### Send digest message via email

In [ ]:
logging.info("----- Start communucation with SMTP Server -----")

try:
        
    gmail = MailServerSSL(BARROCO_SMTP, BARROCO_EMAIL_PORT, BARROCO_ACCOUNT_PASSWORD, BARROCO_EMAIL, BARROCO_EMAIL_NAME)

    message=f"""
    Dear all,
    
    Thanks for following our team digest!
    
    Todays ETL script was runned successfully:
    Rows extracted by API: {rows_income_cnt}
    Rows validated: {rows_transformed_cnt}
    Rows prepared to insert to DB: {rows_prepared_cnt}
    New rows inserted to DB: {inserted_rows_cnt}
    
    The dashboard Google Sheet was also updated successfully.
    
    BR,
    Bamboo team
    """
    logging.info("Email digest message preparing completed successfully")
    
    msg = gmail.send_message('fluxor@atomicmail.io', "Work done", message)
    logging.info("Email digest message was sent successfully")
    
    logging.info("Communucation with SMTP Server completed successfully")

except Exception:
    logging.exception("Communucation with SMTP Server failed")
    raise    

    
print(msg)

Timer stop

In [ ]:
logging.info("----- ETL script finished successfully -----")
logging.info(f"Rows extracted by API: {rows_income_cnt}")
logging.info(f"Rows validated: {rows_transformed_cnt} ")
logging.info(f"Rows prepared to insert to DB: {rows_prepared_cnt}")
logging.info(f"New rows inserted to DB: {inserted_rows_cnt} ")

extract_seconds = round((datetime.fromisoformat(DOWNLOAD_TIME_END) - datetime.fromisoformat(DOWNLOAD_TIME_START)).total_seconds())
extracted_periods = get_periods(extract_seconds)

e_seconds = f"{round(extracted_periods['s'], 2)} seconds"
e_minutes = '' if extracted_periods['m']==0 else f"{extracted_periods['m']} minutes, "
e_hours= '' if extracted_periods['h']==0 else f"{extracted_periods['h']} hours, "
e_days= '' if extracted_periods['d']==0 else f"{extracted_periods['d']} days, "
logging.info(f"API Extracted period: {e_days}{e_hours}{e_minutes}{e_seconds}")


finish_script_seconds = perf_counter() - start_script_time
finish_periods = get_periods(finish_script_seconds)

f_seconds = f"{round(finish_periods['s'], 2)} seconds"
f_minutes = '' if finish_periods['m']==0 else f"{finish_periods['m']} minutes, "
f_hours= '' if finish_periods['h']==0 else f"{finish_periods['h']} hours, "
f_days= '' if finish_periods['d']==0 else f"{finish_periods['d']} days, "
logging.info(f"Script total run time: {f_days}{f_hours}{f_minutes}{f_seconds}")